# CLIP Model Registration for Databricks Model Serving

This notebook demonstrates how to:
- Create a custom MLflow PyFunc wrapper for OpenAI's CLIP model
- Handle both image and text embeddings through a unified interface
- Register the model to Unity Catalog for production use
- Deploy the model to a Databricks Model Serving endpoint

The resulting model endpoint can generate embeddings for both images (base64 encoded) and text, enabling semantic similarity searches and multimodal AI applications.

## Import Required Libraries

Import necessary libraries for MLflow model creation, CLIP model handling, and Databricks SDK integration.


In [ ]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import base64
from PIL import Image
from io import BytesIO
import torch

## Define CLIP Model Wrapper Class

Create a custom MLflow PyFunc class that wraps OpenAI's CLIP model to handle both image and text embeddings through a unified interface.


In [0]:
class CLIPImageTextEmbedding(mlflow.pyfunc.PythonModel):
    
    def load_context(self, context):
        from transformers import CLIPProcessor, CLIPModel
        
        self.model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
        
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
    
    def _get_image_embedding(self, base64_image_str):
        # Remove data URL prefix if present
        if base64_image_str.startswith('data:image'):
            base64_image_str = base64_image_str.split(',')[1]
        
        decoded_bytes = base64.b64decode(base64_image_str)
        image = Image.open(BytesIO(decoded_bytes))
        
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            image_features = self.model.get_image_features(**inputs)
        
        return image_features.cpu().numpy().tolist()[0]
    
    def _get_text_embedding(self, text):
        inputs = self.processor(text=[text], return_tensors="pt", padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            text_features = self.model.get_text_features(**inputs)
        
        return text_features.cpu().numpy().tolist()[0]
    
    def predict(self, context, model_input, params=None):
        input_type = params.get('input_type')
        input_data = model_input['input_data'][0]  # Assuming single string input
        
        if input_type.lower() == 'image':
            return [self._get_image_embedding(input_data)]
        elif input_type.lower() == 'text':
            return [self._get_text_embedding(input_data)]

## Define Model Registration Function

Create a function to register the CLIP model to Unity Catalog with proper MLflow tracking and dependency management.


In [0]:
def register_clip_model(catalog_name="main", schema_name="models", model_name="clip_embedding"):
    mlflow.set_experiment(f"/Shared/clip_embedding_experiment")
    
    with mlflow.start_run() as run:
        # Create model signature
        from mlflow.models import infer_signature
        
        # Sample input for signature inference
        sample_input = pd.DataFrame({"input_data": ["sample text"]})
        sample_params = {"input_type": "text"}
        
        # Load model to generate sample output
        model = CLIPImageTextEmbedding()
        model.load_context(None)
        sample_output = model.predict(None, sample_input, sample_params)
        
        signature = infer_signature(sample_input, sample_output, params=sample_params)
        
        requirements = [
            "torch>=1.9.0",
            "transformers>=4.21.0",
            "Pillow>=8.3.0"
        ]
        
        mlflow.pyfunc.log_model(
            artifact_path="clip_model",
            python_model=model,
            signature=signature,
            pip_requirements=requirements,
            registered_model_name=f"{catalog_name}.{schema_name}.{model_name}"
        )
        
        return f"runs:/{run.info.run_id}/clip_model"

## Define Serving Endpoint Creation Function

Create a function to deploy the registered model to a Databricks Model Serving endpoint for real-time inference.

In [1]:
def create_serving_endpoint(model_name, endpoint_name="clip-embedding-endpoint"):
    from databricks.sdk import WorkspaceClient
    from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput
    
    w = WorkspaceClient()
    
    w.serving_endpoints.create(
        name=endpoint_name,
        config=EndpointCoreConfigInput(
            served_entities=[
                ServedEntityInput(
                    entity_name=model_name,
                    entity_version="1",
                    workload_size="Small",
                    scale_to_zero_enabled=True,
                )
            ]
        )
    )

## Dataset Configuration

Configure the model registration parameters. Update these values to match your Unity Catalog setup.

In [2]:
# Dataset Configuration - Update these values for your setup
CATALOG_NAME = "autobricks"  # Your Unity Catalog name
SCHEMA_NAME = "agriculture"   # Your schema name
MODEL_NAME = "clip_embedding-356"  # Your model name (make it unique)
ENDPOINT_NAME = "clip-embedding-endpoint-356"  # Your endpoint name (make it unique)

NameError: name 'register_clip_model' is not defined

In [ ]:
# Register model to Unity Catalog
model_uri = register_clip_model(CATALOG_NAME, SCHEMA_NAME, MODEL_NAME)

# Create serving endpoint
full_model_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{MODEL_NAME}"
create_serving_endpoint(full_model_name, ENDPOINT_NAME)

print(f"Model registered as: {full_model_name}")
print(f"Serving endpoint: {ENDPOINT_NAME}")


## Usage Examples

Example payload formats for calling the deployed CLIP model endpoint with both text and image inputs.

In [0]:
# Text embedding example
text_payload = {
    "inputs": {"input_data": ["Hello world"]},
    "params": {"input_type": "text"}
}

# Image embedding example (base64 encoded image)
image_payload = {
    "inputs": {"input_data": ["data:image/jpeg;base64,/9j/4AAQ..."]},
    "params": {"input_type": "image"}
}

print("Use these payload formats when calling the model serving endpoint:")
print("Text embedding:", text_payload)
print("Image embedding:", image_payload)